In [2]:
import ee
import geemap

In [3]:
ee.Initialize()

In [12]:
# Seleccionar una imagen Sentinel-2 a partir de su ID
img = ee.Image("COPERNICUS/S2/20230122T141711_20230122T142633_T20KMF")

# Cargar un asset Feature Collection
fc = ee.FeatureCollection("projects/ee-bismarksr17/assets/LOTES_TEST_NUBES")

Map.centerObject(fc.geometry(), 14)

Map.addLayer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Image Example')
Map.addLayer(fc, {}, 'Asset Example FC')
Map

Map(bottom=1150892.0, center=[-17.228494724095913, -62.98520033189894], controls=(WidgetControl(options=['posi…

In [10]:
def cloud_percentage(img, fc):
    # Seleccionar banda de nubes
    cloud = img.select('QA60')
    
    # Calcular la cantidad de pixels con nubes
    cloud_pixels = cloud.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=fc.geometry(),
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    # Calcular la cantidad total de pixels
    total_pixels = cloud.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=fc.geometry(),
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    # Calcular porcentaje de nubes
    cloud_sum = cloud_pixels['QA60']
    total_count = total_pixels['QA60']
    cloud_percentage = cloud_sum / total_count * 100
    
    return cloud_percentage

In [13]:
cloud_percentage(img, fc)

0.0